In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = Image.open(img_path).convert('L')  # Convert to grayscale
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor()
])

# Load dataset
dataset = CustomImageDataset(
    annotations_file='labels.csv',
    img_dir='',
    transform=transform
)

# Split dataset into training and testing set
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=201)
test_dataset, val_dataset = train_test_split(test_dataset, test_size=0.5, random_state=2)  # Further split for validation

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)


In [3]:
len(train_dataset),len(test_dataset),len(val_dataset)

(26824, 3353, 3354)

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class BreastCancerModel(nn.Module):
    def __init__(self):
        super(BreastCancerModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.2)
        self.flatten = nn.Flatten()

        # 3 maxpool lag, 150 / 2 / 2 / 2 = 18
        self.fc2 = nn.Linear(32 * 18 * 18, 1)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = F.relu(self.conv5(x))
        x = self.pool(F.relu(self.conv6(x)))
        x = F.relu(self.conv7(x))
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = BreastCancerModel().to(device)


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
weight_for_positives = 15
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([weight_for_positives], device=device))



num_epochs = 100

In [11]:
for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).unsqueeze(1).type(torch.float32)
        optimizer.zero_grad()
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        predicted = torch.sigmoid(outputs) > 0.5
        
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total

    # Validation phase
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device).unsqueeze(1).type(torch.float32)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            predicted = torch.sigmoid(outputs) > 0.5
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/train_total:.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {val_loss/val_total:.4f}, Val Accuracy: {val_accuracy:.2f}%')

Epoch 1/100, Train Loss: 0.0100, Train Accuracy: 82.33%, Val Loss: 0.0137, Val Accuracy: 79.87%
Epoch 2/100, Train Loss: 0.0095, Train Accuracy: 83.10%, Val Loss: 0.0123, Val Accuracy: 80.17%
Epoch 3/100, Train Loss: 0.0092, Train Accuracy: 83.76%, Val Loss: 0.0157, Val Accuracy: 80.71%
Epoch 4/100, Train Loss: 0.0089, Train Accuracy: 83.88%, Val Loss: 0.0148, Val Accuracy: 85.24%
Epoch 5/100, Train Loss: 0.0087, Train Accuracy: 84.76%, Val Loss: 0.0149, Val Accuracy: 81.75%
Epoch 6/100, Train Loss: 0.0085, Train Accuracy: 84.71%, Val Loss: 0.0149, Val Accuracy: 82.59%
Epoch 7/100, Train Loss: 0.0084, Train Accuracy: 85.14%, Val Loss: 0.0148, Val Accuracy: 84.14%
Epoch 8/100, Train Loss: 0.0085, Train Accuracy: 85.24%, Val Loss: 0.0136, Val Accuracy: 81.72%
Epoch 9/100, Train Loss: 0.0080, Train Accuracy: 85.96%, Val Loss: 0.0190, Val Accuracy: 85.87%
Epoch 10/100, Train Loss: 0.0074, Train Accuracy: 87.13%, Val Loss: 0.0163, Val Accuracy: 85.96%
Epoch 11/100, Train Loss: 0.0075, Train

In [12]:
torch.save(model.state_dict(), 'Cancer_vægt15.pth')

In [24]:
model.load_state_dict(torch.load("breast_cancer_model3.pth"))

<All keys matched successfully>

In [25]:
model.eval()
test_loss, correct, total, false_positives, false_negatives = 0, 0, 0, 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device).unsqueeze(1).type(torch.float32)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        predicted = torch.sigmoid(outputs) > 0.5
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Calculate False Positives and False Negatives
        false_positives += ((predicted == 1) & (labels == 0)).sum().item()
        false_negatives += ((predicted == 0) & (labels == 1)).sum().item()

print(f'Test Accuracy: {100 * correct / total}%')
print(f'False Positives: {false_positives}')
print(f'False Negatives: {false_negatives}')


Test Accuracy: 92.18610199821056%
False Positives: 71
False Negatives: 191
